In [7]:
import os
from glob import glob                  # glob uses the wildcard pattern to create an iterable object file names 
                                       # containing all matching file names in the current directory.
import pandas as pd 
import numpy as np                     # For mathematical calculations 
import seaborn as sns                  # For data visualization 
import matplotlib.pyplot as plt        # For plotting graphs 
%matplotlib inline 
import warnings                        # To ignore any warnings warnings.filterwarnings("ignore")

In [ ]:
def main():
    # read labled training data
    train_label_df = pd.read_csv("./../dataset/train.csv")
    
    # files: to hold list of all datafiles to be read 
    files=['Subject02']
#     for sub in train_label_df.Datafile:
#         files.append(sub.split('/')[0])
        
    print(files)

    for file in set(files):
        # read all the training datasets one by one for each subject
        train_data_df = fetch_train_data(file)
    
    print(type(train_data_df))

# Here is the function to fecth the summarised training data of each subject
def fetch_train_data(subject):
    dataframes = []
    # filenames: holds all the activity files given subject
    file_names = glob("./../dataset/train/" + subject +"/*.csv")
    
    #read each activity file of the subject
    for file_name in file_names:
        df = pd.read_csv(file_name)
        
        #feature extraction of each activity file
        
        processed_df = extract_features(df)
        
        dataframes.append(processed_df)
    
    return dataframes
        
def extract_features(df):

    from scipy.stats import kurtosis
    from scipy.stats import skew

    # time_stats = {'mean','median','min','max','std','variance','mad','rms','zcr','iqr','pe','kurtosis','skew','sma'}
    stats_df = pd.DataFrame()

    # fetch all the features listed in time_stats and append to stats_df

    stats_df['mean'] = df.mean()
    stats_df['median'] = df.median()
    stats_df['min'] = df.min()
    stats_df['max'] = df.max()
    stats_df['std'] = df.std()
    stats_df['variance'] = df.var()
    stats_df['mad'] = df.mad()
    stats_df['rms'] = np.sqrt(np.sum(np.power((df),2))/len(df))
    '''zero-crossing rate is the rate of sign-changes along a signal,
    i.e., the rate at which the signal changes from positive to zero to negative or from negative to zero to positive.
    a = np.array([-2, -1, 0, 1, 2, -2])
    zero_crossings = np.where(np.diff(np.signbit(a)))
    using numpy.signbit() is a little bit quicker than numpy.sign(), since it's implementation is simpler
    However there is one drawback,: If your input array starts and stops with zeros, it will find a zero crossing at the beginning, but not at the end...
    '''
    stats_df['zcr'] = np.diff(np.signbit(df)).sum()
    stats_df['iqr'] = df.quantile(0.75) - df.quantile(0.25)
    # PE is nothing but 75th percentile
    stats_df['pe'] = df.quantile(0.75)
    stats_df['kurtosis'] = kurtosis(df)
    stats_df['skew'] = skew(df)
    # stats_df['sma'] = df.

    return(stats_df.transpose())
    
    
if __name__ == '__main__':
    main()

In [45]:
def fetch_train_data(subject):
    dataframes = []
    filenames = glob("./../dataset/train/" + subject +"/*.csv")
    
    for file_name in file_names:
        df = pd.read_csv(file_name)
        
    
fetch_train_data('Subject02')

./../dataset/train/Subject02\Subject02_Aufnahme002.csv


In [103]:
from math import sqrt
from scipy.stats import kurtosis
from scipy.stats import skew

df = pd.read_csv('./../dataset/train/Subject02\Subject02_Aufnahme002.csv', header=None)

# time_stats = {'mean','median','min','max','std','variance','mad','rms','zcr','iqr','pe','kurtosis','skew','sma'}
# stats_df = pd.DataFrame()
stats_df['mean'] = df.mean()
stats_df['median'] = df.median()
stats_df['min'] = df.min()
stats_df['max'] = df.max()
stats_df['std'] = df.std()
stats_df['variance'] = df.var()
stats_df['mad'] = df.mad()
stats_df['rms'] = np.sqrt(np.sum(np.power((df),2))/len(df))
'''zero-crossing rate is the rate of sign-changes along a signal,
i.e., the rate at which the signal changes from positive to zero to negative or from negative to zero to positive.
a = np.array([-2, -1, 0, 1, 2, -2])
zero_crossings = np.where(np.diff(np.signbit(a)))
using numpy.signbit() is a little bit quicker than numpy.sign(), since it's implementation is simpler
However there is one drawback,: If your input array starts and stops with zeros, it will find a zero crossing at the beginning, but not at the end...
'''
stats_df['zcr'] = np.diff(np.signbit(df)).sum()
stats_df['iqr'] = df.quantile(0.75) - df.quantile(0.25)
# PE is nothing but 75th percentile
stats_df['pe'] = df.quantile(0.75)
stats_df['kurtosis'] = kurtosis(df)
stats_df['skew'] = skew(df)
# stats_df['sma'] = df.

print(stats_df.transpose())

                    0              1             2             3   \
mean      3.276542e+04   32756.443801  32757.803785  33161.544998   
median    3.276900e+04   32827.000000  32758.000000  33167.000000   
min       2.244700e+04   25719.000000  31796.000000  32507.000000   
max       4.446000e+04   37326.000000  34491.000000  33719.000000   
std       1.386335e+03     908.594096    201.695685    140.639374   
variance  1.921925e+06  825543.231481  40681.149275  19779.433542   
mad       6.328083e+02     594.832451    138.814427    110.452872   
rms       3.279472e+04   32769.037747  32758.424478  33161.843110   
zcr       0.000000e+00       0.000000      0.000000      0.000000   
iqr       2.010000e+02     731.000000    198.000000    188.000000   
pe        3.285700e+04   33152.000000  32855.000000  33257.000000   
kurtosis  1.332107e+01       7.626903      6.769380      0.965453   
skew      4.194823e-01      -0.974566      0.512606     -0.270198   

                     4           